In [12]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (239 kB/s)
Reading package lists... Done


In [13]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-05 23:33:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.51MB/s    in 0.2s    

2023-04-05 23:33:24 (5.51 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [15]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok|2015-08-31 00:00:00|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...| 

In [16]:
# retrieve all the rows where the total_votes count is equal to or greater than 20
sample_df = df.filter('total_votes>20')
sample_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3200070|R3FJ319XA6ZAUQ|B00ZI7IH1E|     450586100|Funlux? 8CH Full ...|          Camera|          2|           21|         25|   N|                Y|            Mediocre|The product is gr...|2015-08-31 00:00:00|
|         US|   40768960| R6HRF25HUMIIE|B00TAG8F52|     482310914|LimoStudio Digita...| 

In [20]:
# Get row count prior to creating newSample_df
rows = sample_df.count()
print(rows)

55966


In [21]:
newSample_df = sample_df.filter(sample_df["helpful_votes"]/sample_df["total_votes"] >= 0.5)
newSample_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3200070|R3FJ319XA6ZAUQ|B00ZI7IH1E|     450586100|Funlux? 8CH Full ...|          Camera|          2|           21|         25|   N|                Y|            Mediocre|The product is gr...|2015-08-31 00:00:00|
|         US|   40768960| R6HRF25HUMIIE|B00TAG8F52|     482310914|LimoStudio Digita...| 

In [22]:
# Get row count priot to creating newSample_df
rows = newSample_df.count()
print(rows)

48283


In [23]:
# Filter all results where vine == "Y"
vine_yes_df = newSample_df.filter(newSample_df.vine=='Y')
vine_yes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   38607681|R37L6YZDX0P3J9|B00SEKXJ6U|     408705341|Guardzilla  All-I...|          Camera|          1|           28|         35|   Y|                N|Gave up after an ...|I have installed ...|2015-08-29 00:00:00|
|         US|   52532594|R2JT4XRQO2KASL|B00SEKXJ6U|     408705341|Guardzilla  All-I...| 

In [24]:
# Filter all results where vine == "N"
vine_no_df = newSample_df.filter(newSample_df.vine=='N')
vine_no_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3200070|R3FJ319XA6ZAUQ|B00ZI7IH1E|     450586100|Funlux? 8CH Full ...|          Camera|          2|           21|         25|   N|                Y|            Mediocre|The product is gr...|2015-08-31 00:00:00|
|         US|   40768960| R6HRF25HUMIIE|B00TAG8F52|     482310914|LimoStudio Digita...| 

In [35]:
# Numbers for the vine = yes (paid)
# functions.count()
from pyspark.sql.functions import count
#Count total ratings
rows = vine_yes_df.count()
print("Paid Vine User Reviews:")
print(f"Total ratings : {rows}" )

# Count star rating >=5 
yes_over_five =vine_yes_df.filter('star_rating >=5').count()
print(f"Count of ratings >= 5 : {yes_over_five}")

# Percent of ratings >=5
yes_percent_over = yes_over_five / rows
print(f"Percentage of rating >= 5 : {yes_percent_over}")


Paid Vine User Reviews:
Total ratings : 580
Count of ratings >= 5 : 246
Percentage of rating >= 5 : 0.4241379310344828


In [34]:
# Numbers for the vine = no (unpaid)
rows = vine_no_df.count()
print("Unpaid Reviews:")
print(f"Total ratings : {rows}")

# Count star rating >=5 
no_over_five =vine_no_df.filter('star_rating >=5').count()
print(f"Count of ratings >= 5 : {no_over_five}")

# Percent of ratings >=5
no_percent_over = no_over_five / rows
print(f"Percentage of rating >= 5 : {no_percent_over}")


Unpaid Reviews:
Total ratings : 47703
Count of ratings >= 5 : 23837
Percentage of rating >= 5 : 0.4996960358887282


In [ ]:
newSample_df.filter((newSample_df.vine=='Y') & (newSample_df.verified_purchase=='Y')).count()

5

In [ ]:
newSample_df.filter((newSample_df.vine=='N') & (newSample_df.verified_purchase=='Y')).count()

23722